In [1]:
import pandas as pd
import json
import re
import pymorphy2
import nltk
import string
from datetime import date
from os import listdir
from os.path import isfile, join

In [2]:
cd RU_JSON

/Users/aleksandrsimanychev/Untitled Folder/RU_JSON


In [3]:
onlyfiles = [f for f in listdir() if f.endswith('.json')]

In [4]:
ru_df = pd.DataFrame({'Channel': [1], 'Post': [1], 'Date': [1], 'Time': [1], 'Location': [1], 'Person': [1]})

In [5]:
counter = 0
for file in onlyfiles:
    with open(file, "r") as read_file:
        datas = json.load(read_file)
        for data in datas:
            ru_df.loc[counter, 'Channel'] = file[:-5]
            try:
                ru_df.loc[counter, 'Post'] = data['message']
            except:
                ru_df.loc[counter, 'Post'] = ''
            ru_df.loc[counter, 'Date'] = data['date'][:10]
            ru_df.loc[counter, 'Time'] = data['date'][11:19]
            counter += 1

In [7]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)


In [8]:
for i in range(ru_df['Post'].size):
    LocList = []
    doc = Doc(ru_df.loc[i, 'Post'])
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
        if span.type == 'LOC':
            LocList.append(span.normal)
    ru_df.loc[i, 'Location'] = str(LocList)

In [39]:
ru_df['Location'] = ru_df['Location'].apply(lambda x : re.sub(r"'", '', x))

In [ ]:
ru_df['Location'] = ru_df['Location'].apply(lambda x : re.sub(r"\[", '', x))

In [ ]:
ru_df['Location'] = ru_df['Location'].apply(lambda x : re.sub(r"\]", '', x))

In [14]:
from collections import Counter

In [64]:
all_days_df=pd.DataFrame({'Region' : [], 'Ukrainian Name':[], 'Russian Name':[], 'Longitude_WGS84':[],
       'Latitude_WGS84':[], 'Freq':[], 'Date':[]})

In [18]:
cd ../

/Users/aleksandrsimanychev/Untitled Folder/RU_JSON


In [19]:
RuUaLocDF = pd.read_csv('GeoFile.csv', sep =';')

In [65]:
for date in ru_df['Date'].unique():
    x_day = ', '.join(e for e in ru_df[ru_df['Date'] == date]['Location'].array)
    Counter(x_day.split(', ')).most_common()
    x_day_df = RuUaLocDF.copy(deep=True)
    for place in Counter(x_day.split(', ')).most_common():
        if place[0] in list(x_day_df['Russian Name']):
            x_day_df.loc[x_day_df[x_day_df['Russian Name'] == place[0]].index, 'Freq'] = place[1]
    x_day_df['Date'] = date
    x_day_df = x_day_df.dropna()
    all_days_df = pd.concat([all_days_df, x_day_df])

In [67]:
all_days_df.to_csv('all_days.csv')

In [3]:
cd UA_JSON

/Users/aleksandrsimanychev/Untitled Folder/UA_JSON


In [4]:
onlyfiles_ua = [f for f in listdir() if f.endswith('.json')]

In [5]:
ua_df = pd.DataFrame({'Channel': [1], 'Post': [1], 'Date': [1], 'Time': [1]})

In [7]:
counter = 0
for file in onlyfiles_ua:
    with open(file, "r") as read_file:
        datas = json.load(read_file)
        for data in datas:
            ua_df.loc[counter, 'Channel'] = file[:-5]
            try:
                ua_df.loc[counter, 'Post'] = data['message']
            except:
                ua_df.loc[counter, 'Post'] = ''
            ua_df.loc[counter, 'Date'] = data['date'][:10]
            ua_df.loc[counter, 'Time'] = data['date'][11:19]
            counter += 1

In [9]:
for i in range(ua_df['Post'].size):
    LocList = []
    doc = Doc(ua_df.loc[i, 'Post'])
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
        if span.type == 'LOC':
            LocList.append(span.normal)
    ua_df.loc[i, 'Location'] = str(LocList)

In [11]:
ua_df['Location'] = ua_df['Location'].apply(lambda x : re.sub(r"'", '', x))

In [12]:
ua_df['Location'] = ua_df['Location'].apply(lambda x : re.sub(r"\[", '', x))

In [13]:
ua_df['Location'] = ua_df['Location'].apply(lambda x : re.sub(r"\]", '', x))

In [15]:
all_days_df_ua=pd.DataFrame({'Region' : [], 'Ukrainian Name':[], 'Russian Name':[], 'Longitude_WGS84':[],
       'Latitude_WGS84':[], 'Freq':[], 'Date':[]})

In [20]:
for date in ua_df['Date'].unique():
    x_day = ', '.join(e for e in ua_df[ua_df['Date'] == date]['Location'].array)
    Counter(x_day.split(', ')).most_common()
    x_day_df = RuUaLocDF.copy(deep=True)
    for place in Counter(x_day.split(', ')).most_common():
        if place[0] in list(x_day_df['Russian Name']):
            x_day_df.loc[x_day_df[x_day_df['Russian Name'] == place[0]].index, 'Freq'] = place[1]
    x_day_df['Date'] = date
    x_day_df = x_day_df.dropna()
    all_days_df_ua = pd.concat([all_days_df_ua, x_day_df])

In [21]:
all_days_df_ua.to_csv('all_days_ua.csv')